In [1]:
"""
This code can run without problem.

How to run:
1. Please input network filename:
2. then it does the shortest path calculation by Dijkstra's algorithm (min-heap)
3. then use a while loop to ask the user to input the origin  (say, s)
     if the origin is '0', then the code STOP
     else the code will ask the user to input the destination (say, t),
     and output the shortest distance from s to t and the entire path

This code is written by 王晏國, email h44114025@gs.ncku.edu.tw, on 2023/12/16
"""


import heapq
import re
import numpy as np

# 前處理
def sp_to_txt(sp_file, txt_file):
    with open(sp_file, "r") as sp:
        with open(txt_file, "w") as txt:
            for line in sp:
                if line[0] == "c":
                    continue
                elif line[0] == "t":
                    modified_line = re.sub('\s+', ',', line)
                    txt.write(modified_line + '\n')
                elif line[0] == "a":
                    modified_line = re.sub('\s+', ',', line)
                    txt.write(modified_line + '\n')
                elif line[0] == "p":
                    modified_line = re.sub('\s+', ',', line)
                    txt.write(modified_line + '\n')
                elif line[0] == "n":
                    modified_line = re.sub('\s+', ',', line)
                    txt.write(modified_line + '\n')

def filename():
    with open(txt_file, "r") as file:
        filename = file.readline().split(",")[1]

    return filename

def extract_numbers(line):
    # 使用正則表達式匹配數字
    numbers = re.findall(r'\d+', line)
    # 將匹配到的數字轉換為整數
    numbers = [int(num) for num in numbers]
    return numbers

def sp_to_list(txt_file):
    in_list = []
    # 開啟.sp檔案以讀取模式
    with open(txt_file, 'r') as num_file:
        for line in num_file:
            # 提取每一行的數字並形成串列
            numbers = extract_numbers(line)
            in_list.append(numbers)

    return in_list

"""---------------------------------------------------------------"""
# 完成檔案處理

sp_file = input("Please input network filename: ")
txt_file = "output.txt"
sp_to_txt(sp_file, txt_file)
numbers_list = sp_to_list(txt_file)
# print(numbers_list)

"""---------------------------------------------------------------"""
# Functions

def From_nodes():
    From = np.zeros(numbers_list[1][1])
    for index in range(numbers_list[1][1]):
        From[index] += numbers_list[index+3][0]

    return From


def To_nodes():
    To = np.zeros(numbers_list[1][1])
    for index in range(numbers_list[1][1]):
        To[index] += numbers_list[index + 3][1]

    return To

def Length_list():
    Length = np.zeros(numbers_list[1][1])
    for index in range(numbers_list[1][1]):
        Length[index] += numbers_list[index + 3][2]

    return Length


def Binary_min_heap():  # 要記(length, pred, current)
    From = From_nodes()
    To = To_nodes()
    Length = Length_list()

    heap = [(0, 0, origin)]    # (length, from_node)
    path = []
    visited = []
    inf_of_node = []
    shortest_distance = 0

    while len(heap) != 0:

        (length, pred, current) = heapq.heappop(heap)
        if current == destination:  # 若起點為給定之終點，跳出迴圈
            visited.append(current)
            break

        if current not in visited and current == origin:  # 若現在的起點先前沒當過起點，則開始scan
            visited.append(current)
            inf_of_node.append((length, pred, current))
        elif current not in visited and current != origin:
            visited.append(current)
        elif current in visited:   # 當過起點，則continue
            continue

        for i in range(numbers_list[1][1]):     # 同一個起點的scan
            if int(From[i]) == current:     # scan
                node_set = (int(length + Length[i]), current, int(To[i]))

                for j in range(len(inf_of_node)):   # 判斷是否需要distance update

                    if node_set[2] == inf_of_node[j][2]:  # 若從先前的node資訊內發現inf_of_node[2]和現在scan到的node是同一個

                        if node_set[0] < inf_of_node[j][0]:     # 代表需要做distance label的檢查
                            inf_of_node[j] = node_set   # 若現在累積的length小於原本紀錄的，則把原本的node資訊換成現在的
                            heapq.heappush(heap, node_set)
                            break
                        elif node_set[0] > inf_of_node[j][0]:
                            break

                    elif node_set[2] != inf_of_node[j][2] and j != len(inf_of_node)-1:
                        continue

                    else:
                        inf_of_node.append(node_set)
                        heapq.heappush(heap, node_set)
                        break


    find_pred = 0
    times = 0
    # print(inf_of_node)

    while find_pred != origin:
        if times == 0:
            for i in range(len(inf_of_node)):
                if inf_of_node[i][2] == destination:
                    shortest_distance = inf_of_node[i][0]
                    path.append(inf_of_node[i][2])
                    find_pred = inf_of_node[i][1]
                    # print(inf_of_node)
                    times += 1
        elif times >= 1:
            for j in range(len(inf_of_node)):
                if inf_of_node[j][2] == find_pred:
                    path.append(inf_of_node[j][2])
                    find_pred = inf_of_node[j][1]
    if find_pred == origin:
        path.append(origin)
    #print(path)

    return [path, shortest_distance]

"""---------------------------------------------------------------"""
# 執行

while True:
    order_str = ""

    origin = int(input("Please input the origin (input '0' to stop): "))
    if origin != 0:
        destination = int(input("Please input the destination: "))
        heap_output = Binary_min_heap()

        for index in range(len(heap_output[0])):
            if index != len(heap_output[0])-1:
                order_str += str(heap_output[0][index]) + "<"
            else:
                order_str += str(heap_output[0][index])

        print(f"{origin}-{destination}: [{heap_output[1]}] {order_str}")
    else:
        print("-END-")
        break



Please input network filename:  input_100_400_1.sp
Please input the origin (input '0' to stop):  1
Please input the destination:  2


1-2: [3] 2<1


Please input the origin (input '0' to stop):  2
Please input the destination:  3


2-3: [8] 3<2


Please input the origin (input '0' to stop):  1
Please input the destination:  5


...

1-53: [7] 53<2<1


Please input the origin (input '0' to stop):  31
Please input the destination:  42


31-42: [23] 42<99<67<3<31


Please input the origin (input '0' to stop):  0


-END-
